In [ ]:
import mailbox
import chardet

# Read the contents of the uploaded file into a bytes object
with open('/content/fradulent_emails.txt', 'rb') as f:
    data = f.read()

# Detect the encoding of the file using chardet
result = chardet.detect(data)
encoding = result['encoding']

# Decode the file contents using the detected encoding
email_lines = data.decode(encoding).splitlines()

# Create a new mailbox file
mbox = mailbox.mbox('output.mbox')

# Initialize variables for storing email headers and body
headers = {}
body = []

# Iterate through the lines in the text file
for line in email_lines:
    # If the line starts with "From ", it indicates a new email
    if line.startswith('From '):
        # If there is already an email being processed, add it to the mbox file
        if headers:
            # Combine the email headers and body into a single string
            email = '\n'.join([f'{k}: {v}' for k, v in headers.items()] + [''] + body)
            # Add the email to the mbox file
            mbox.add(mailbox.mboxMessage(email.encode('utf-8')))
            # Reset the variables for the next email
            headers = {}
            body = []
        # Parse the email headers from the "From " line
        headers['From'] = line[5:].strip()
    # If the line starts with a header field name followed by a colon, add it to the headers dictionary
    elif ':' in line:
        field, value = line.split(':', 1)
        headers[field.strip()] = value.strip()
    # Otherwise, add the line to the email body
    else:
        body.append(line.strip())

# If there is any email left to be processed, add it to the mbox file
if headers:
    email = '\n'.join([f'{k}: {v}' for k, v in headers.items()] + [''] + body)
    mbox.add(mailbox.mboxMessage(email.encode('utf-8')))

# Close the mbox file
mbox.close()

In [ ]:
import re
import os
import nltk
import string
import mailbox
import pandas as pd
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from email.header import decode_header
from nltk.tokenize import word_tokenize
import nltk
from urllib.parse import urlparse
!pip install whois
import whois
import requests
from datetime import datetime
!pip install tldextract
import tldextract
import time
import urllib.parse

# Download the stopwords resource
nltk.download('stopwords')

nltk.download('punkt')

  Preparing metadata (setup.py) ... done
  Created wheel for whois: filename=whois-0.9.27-py3-none-any.whl size=30469 sha256=53d4d74b69cb28366c75747e0b63e4e343de51d29586d8acc934309cf26f4110
  Stored in directory: /root/.cache/pip/wheels/71/17/36/d62eb5bcc416650499a7259d584c11e4a778de5ce0e72a8dbf
Successfully built whois
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 1.4 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
class EmailParser:
    urlRegex = r'https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b(?:[-a-zA-Z0-9()@:%_\+.~#?&\/=;]*)'
    emailRegex = r'([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)'

    def __init__(self, email):
        self.email = email
        self.__extract_email_parts()

    def __extract_email_parts(self):
        no_of_attachments = 0
        text = str(self.email['Subject']) + " "
        htmlDoc = ""
        for part in self.email.walk():
            content_type = part.get_content_type()
            if content_type == 'text/plain':
                text += str(part.get_payload())
            elif content_type == 'text/html':
                htmlDoc += part.get_payload()
            else:
                main_content_type = part.get_content_maintype()
                if main_content_type in ['image','application']:
                    no_of_attachments += 1
        self.text, self.html, self.no_of_attachments = text, htmlDoc, no_of_attachments

    def get_urls(self):
        text_urls = set(re.findall(EmailParser.urlRegex,self.text))
        html_urls = set(re.findall(EmailParser.urlRegex,self.html))
        return list(text_urls.union(html_urls))

    def get_email_text(self):
        if(self.html != ""):
            soup = BeautifulSoup(self.html)
            self.text += soup.text
        return self.text

    def get_no_of_attachments(self):
        return self.no_of_attachments

    def get_sender_email_address(self):
        sender = email['From']
        try:
            emails = re.findall(EmailParser.emailRegex, sender)
        except:
            h = decode_header(email['From'])
            header_bytes = h[0][0]
            sender = header_bytes.decode('ISO-8859-1')
            emails = re.findall(EmailParser.emailRegex, sender)
        if(len(emails) != 0):
            return emails[len(emails)-1]
        else:
            return ''


In [ ]:
class StringUtil:

    dotRegex = r'\.'
    digitsRegex = r'[0-9]'
    ipAddressRegex = r'(?:[0-9]{1,3}\.){3}[0-9]{1,3}'
    dashesRegex = r'-'
    specialCharsRegex = r'[()@:%_\+~#?\=;]'
    words = Counter()
    stop_words = set(stopwords.words('english'))
    stemmer = nltk.PorterStemmer()
    punctuations = ['!','@','#','$','%','^','&','*','(',')','-','_','=','+',';',':',"'",'"','?','/','<','>','.',',','/','~','`']
    def process_text(self, text):
        text = text.lower()                    #lowercase
        text = re.sub(r'[\n\t\r]', ' ', text)  #remove escape sequences

        #remove punctuations
        punctuation = string.punctuation  # Get all punctuation marks
        translator = str.maketrans('', '', punctuation + string.digits)  # Create a translator to remove punctuation and digits
        text = text.translate(translator)  # Remove punctuation and digits using translate()

        #tokenize and stem words
        word_tokens = word_tokenize(text)
        filtered_text = []
        for w in word_tokens:
            if w not in StringUtil.stop_words:
                filtered_text.append(w)

        #count frequency of words
        word_counts = Counter(filtered_text)
        stemmed_word_count = Counter()
        for word, count in word_counts.items():
            stemmed_word = StringUtil.stemmer.stem(word)
            stemmed_word_count[stemmed_word] += count
        word_counts = stemmed_word_count
        StringUtil.words += word_counts
        return word_counts
    def get_most_common_words(self):
        return StringUtil.words.most_common(1000)
    def process_email_address(self, emailid):
        length, noOfDots, noOfDashes, noOfSpecialChars, noOfDigits, noOfSubdomains = 0,0,0,0,0,0

        length = len(emailid)
        if(length > 0):
            username, domain = emailid.split('@')
            noOfSubdomains = len(re.findall(StringUtil.dotRegex,domain)) - 1
            noOfDots = len(re.findall(StringUtil.dotRegex, username))
            noOfSpecialChars = len(re.findall(StringUtil.specialCharsRegex, username))
            noOfDashes = len(re.findall(StringUtil.dashesRegex, emailid))
            noOfDigits = len(re.findall(StringUtil.digitsRegex, emailid))
        return length, noOfDots, noOfDashes, noOfSpecialChars, noOfDigits, noOfSubdomains
    def process_urls(self,urls):
        api_key = '43ef6ac4c119080b1c8a0eff0ba430e99cef1f11960d3c2200b7b42ae05963a6'
        features = {}
        if (len(urls) == 0):
         features['noof_urls'] = 0
         features['length_url'] = 0
         features['length_hostname'] = 0
         features['ip'] = 0
         features['ip_asn'] = 0
         features['ip_last_analysis_date'] = 0
         features['ip_harmless'] = 0
         features['ip_malicious'] = 0
         features['ip_suspicious'] = 0
         features['ip_timeout'] = 0
         features['ip_undetected'] = 0
         features['ip_last_modification_date'] = 0
         features['ip_harmless_votes'] = 0
         features['ip_malicious_votes'] = 0
         features['first_submission_date'] =  0
         features['last_analysis_date'] = 0
         features['harmless'] = 0
         features['malicious'] = 0
         features['suspicious'] = 0
         features['timeout'] = 0
         features['undetected'] = 0
         features['last_http_response_code'] = 0
         features['last_http_response_content_length'] = 0
         features['last_modification_date'] = 0
         features['harmless_votes'] = 0
         features['malicious_votes'] = 0
         features[' dots_number'] =  0
         features['hyphens_number'] = 0
         features['at_number'] = 0
         features['qm_number'] = 0
         features['and_number'] = 0
         features['or_number'] = 0
         features['eq_number'] = 0
         features['tildes_number'] = 0
         features['percent_number'] = 0
         features['slash_number'] = 0
         features['star_number'] =  0
         features['colon_number'] = 0
         features['comma_number'] = 0
         features['semicolon_number'] = 0
         features['dollar_number'] = 0
         features['space_number'] = 0
         features['www_number'] =  0
         features['com_number'] = 0
         features['http_in_path'] = 0
         features['https_token'] = 0
         features['ratio_digits_url'] = 0
         features['ratio_digits_host'] = 0
         features['punycode'] = 0
         features['port'] =  0
         features['tld_in_path'] = 0
         features['tld_in_subdomain'] = 0
         features['abnormal_subdomain'] = 0
         features['subdomains_number'] = 0
         features['prefix_suffix'] = 0
         features['random_domain'] = 0
         features['shortening_service'] = 0
         features['path_extension'] = 0
         features['redirections_number'] = 0
         features['external_redirections_number'] = 0
         features['length_words_raw'] = 0
         features['char_repeat'] = 0
         features['shortest_word_host'] = 0
         features['longest_word_host'] =0
         features['shortest_word_path'] = 0
         features['longest_word_path'] = 0
         features['shortest_words_raw'] = 0
         features['longest_words_raw'] = 0
         features['average_word_raw'] = 0
         features['average_word_host'] = 0
         features['average_word_path'] = 0
         features['phish_hints'] =  0
         features['hyperlinks_number'] = 0
         features['ratio_intHyperlinks'] = 0
         features['ratio_extHyperlinks'] = 0
         features['ratio_nullHyperlinks'] = 0
         features['extCSS_number'] = 0
         features['submit_emails'] = 0
         features['ratio_intMedia'] = 0
         features['ratio_extMedia'] = 0
         features['sfh'] = 0
        else:
          features['noof_urls'] = len(urls)
          for url in urls:
           # Extract the URL components
            url_parts = re.findall('https?://([^/]+)([^?#]*)(\\?[^#]*)?(#.*)?', url)
            if len(url_parts) == 0:
                continue
            host = url_parts[0][0]
            path = url_parts[0][1]
            query = url_parts[0][2] if len(url_parts[0]) > 2 else ''
            fragment = url_parts[0][3] if len(url_parts[0]) > 3 else ''

            # Extract the URL features
            length_url = len(url)
            features['length_url'] = length_url
            length_hostname = len(host)
            features['length_hostname'] = length_hostname
            ip = 1 if re.match('^(?:[0-9]{1,3}\.){3}[0-9]{1,3}$', host) else 0
            features['ip'] =  ip
            try:
               ip_address = urllib.parse.urlparse(url).hostname
            # Make API request to retrieve IP address information
               params = {'ip': ip_address, 'apikey': api_key}
               response = requests.get('https://www.virustotal.com/api/v3/ip_addresses', params=params)

               ip_info = response.json()['data'][0]['attributes']
               features['ip_asn'] = ip_info['asn']
               features['ip_last_analysis_date'] = ip_info['last_analysis_date']
               features['ip_harmless'] = ip_info['last_analysis_stats']['harmless']
               features['ip_malicious'] = ip_info['last_analysis_stats']['malicious']
               features['ip_suspicious'] = ip_info['last_analysis_stats']['suspicious']
               features['ip_timeout'] = ip_info['last_analysis_stats']['timeout']
               features['ip_undetected'] = ip_info['last_analysis_stats']['undetected']
               features['ip_last_modification_date'] = ip_info['last_modification_date']
               features['ip_harmless_votes'] = ip_info['total_votes']['harmless']
               features['ip_malicious_votes'] = ip_info['total_votes']['malicious']

            except:
                 features['ip_asn'] = 0
                 features['ip_last_analysis_date'] = 0
                 features['ip_harmless'] = 0
                 features['ip_malicious'] = 0
                 features['ip_suspicious'] = 0
                 features['ip_timeout'] = 0
                 features['ip_undetected'] = 0
                 features['ip_last_modification_date'] = 0
                 features['ip_harmless_votes'] = 0
                 features['ip_malicious_votes'] = 0
            try:
               url = "https://www.virustotal.com/vtapi/v2/url/report"
               params = {"apikey": api_key , "resource": url}
               response = requests.get(url, params=params)
               url_info = response.json()['data'][0]['attributes']
               features['first_submission_date'] =  url_info['first_submission_date']
               features['last_analysis_date'] = url_info['last_analysis_date']
               features['harmless'] = url_info['last_analysis_stats']['harmless']
               features['malicious'] = url_info['last_analysis_stats']['malicious']
               features['suspicious'] = url_info['last_analysis_stats']['suspicious']
               features['timeout'] = url_info['last_analysis_stats']['timeout']
               features['undetected'] = url_info['last_analysis_stats']['undetected']
               features['last_http_response_code'] = url_info['last_http_response_code']
               features['last_http_response_content_length'] = url_info['last_http_response_content_length']
               features['last_modification_date'] = url_info['last_modification_date']
               features['harmless_votes'] = url_info['total_votes']['harmless']
               features['malicious_votes'] = url_info['total_votes']['malicious']
            except:
               features['first_submission_date'] =  0
               features['last_analysis_date'] = 0
               features['harmless'] = 0
               features['malicious'] = 0
               features['suspicious'] = 0
               features['timeout'] = 0
               features['undetected'] = 0
               features['last_http_response_code'] = 0
               features['last_http_response_content_length'] = 0
               features['last_modification_date'] = 0
               features['harmless_votes'] = 0
               features['malicious_votes'] = 0
            dots_number = host.count('.')
            features[' dots_number'] =   dots_number
            hyphens_number = host.count('-')
            features['hyphens_number'] = hyphens_number
            at_number = host.count('@')
            features['at_number'] = at_number
            qm_number = query.count('?')
            features['qm_number'] = qm_number
            and_number = query.count('&')
            features['and_number'] = and_number
            or_number = query.count('|')
            features['or_number'] = or_number
            eq_number = query.count('=')
            features['eq_number'] = eq_number
            tildes_number = query.count('~')
            features['tildes_number'] =  tildes_number
            percent_number = query.count('%')
            features['percent_number'] =  percent_number
            slash_number = path.count('/')
            features['slash_number'] =  slash_number
            star_number = path.count('*')
            features['star_number'] =  star_number
            colon_number = path.count(':')
            features['colon_number'] =  colon_number
            comma_number = path.count(',')
            features['comma_number'] =  comma_number
            semicolon_number = path.count(';')
            features['semicolon_number'] = semicolon_number
            dollar_number = path.count('$')
            features['dollar_number'] = dollar_number
            space_number = path.count(' ')
            features['space_number'] = space_number
            www_number = 1 if 'www' in host else 0
            features['www_number'] =  www_number
            com_number = 1 if '.com' in host else 0
            features['com_number'] =  com_number
            http_in_path = 1 if 'http' in path else 0
            features['http_in_path'] =   http_in_path
            https_token = 1 if re.search('https', url) else 0
            features['https_token'] =  https_token
            ratio_digits_url = sum(c.isdigit() for c in url) / len(url)
            features['ratio_digits_url'] =  ratio_digits_url
            ratio_digits_host = sum(c.isdigit() for c in host) / len(host)
            features['ratio_digits_host'] =  ratio_digits_host
            punycode = 1 if 'xn--' in host else 0
            features['punycode'] =  punycode
            port = 1 if ':' in host else 0
            features['port'] =  port
            tld_in_path = 1 if re.search('\.[a-z]{2,3}/', url) else 0
            features['tld_in_path'] = tld_in_path
            tld_in_subdomain = 1 if re.search('\.[a-z]{2,3}\.', host) else 0
            features['tld_in_subdomain'] = tld_in_subdomain
            abnormal_subdomain = 1 if re.search('[^a-zA-Z0-9.-]', host) else 0
            features['abnormal_subdomain'] = abnormal_subdomain
            subdomains_number = host.count('.')
            features['subdomains_number'] = subdomains_number
            prefix_suffix = 1 if re.match('^[^@]+@[^@]+\.[^@]+$', url) else 0
            features['prefix_suffix'] = prefix_suffix
            random_domain = 1 if re.search('[0-9]{5,}', host) else 0
            features['random_domain'] = random_domain
            shortening_service = 1 if re.search('bit\.ly|goo\.gl|ow\.ly|t\.co|tinyurl\.', url) else 0
            features['shortening_service'] = shortening_service
            path_extension = 1 if re.search('\.[a-zA-Z]{3,4}', path) else 0
            features['path_extension'] = path_extension
            # Send HTTP request to webpage
            try:
                  response = requests.get(url, timeout=5)
                  response.raise_for_status()
            except requests.exceptions.Timeout:
             features['redirections_number'] = 0
             features['external_redirections_number'] = 0
             features['phish_hints'] =  0
             features['hyperlinks_number'] = 0
             features['ratio_intHyperlinks'] = 0
             features['ratio_extHyperlinks'] = 0
             features['ratio_nullHyperlinks'] = 0
             features['extCSS_number'] = 0
             features['submit_emails'] = 0
             features['ratio_intMedia'] = 0
             features['ratio_extMedia'] = 0
             features['sfh'] = 0
             #continue
            except requests.exceptions.HTTPError as error:
             features['redirections_number'] = 0
             features['external_redirections_number'] = 0
             features['phish_hints'] =  0
             features['hyperlinks_number'] = 0
             features['ratio_intHyperlinks'] = 0
             features['ratio_extHyperlinks'] = 0
             features['ratio_nullHyperlinks'] = 0
             features['extCSS_number'] = 0
             features['submit_emails'] = 0
             features['ratio_intMedia'] = 0
             features['ratio_extMedia'] = 0
             features['sfh'] = 0
             #continue
            except requests.exceptions.RequestException as error:
             features['redirections_number'] = 0
             features['external_redirections_number'] = 0
             features['phish_hints'] =  0
             features['hyperlinks_number'] = 0
             features['ratio_intHyperlinks'] = 0
             features['ratio_extHyperlinks'] = 0
             features['ratio_nullHyperlinks'] = 0
             features['extCSS_number'] = 0
             features['submit_emails'] = 0
             features['ratio_intMedia'] = 0
             features['ratio_extMedia'] = 0
             features['sfh'] = 0
            #continue
            # Count number of redirections
            try:
               redirections_number = len(response.history)
               features['redirections_number'] = redirections_number
            except:
               features['redirections_number'] = 0
            # Count number of external redirections
            try:
               external_redirections_number = 0
               for resp in response.history:
                 url_parsed = urlparse(resp.url)
                 if url_parsed.netloc != urlparse(url).netloc:
                      external_redirections_number += 1
               features['external_redirections_number'] = external_redirections_number
            except:
                    features['external_redirections_number'] = 0
            # Extract domain and paths
            url_parsed = urlparse(url)
            domain = url_parsed.netloc
            path = url_parsed.path

            # Extract words from domain and paths
            domain_words = re.findall(r'\w+', domain)
            path_words = re.findall(r'\w+', path)

            # Calculate length of raw URL and number of characters repeated
            try:
               length_words_raw = len(re.findall(r'\w+', url))
               char_repeat = len(re.findall(r'(.)\1{2,}', url))
            except:
                 length_words_raw = 0
                 char_repeat = 0
            features['char_repeat'] = char_repeat
            features['length_words_raw'] = length_words_raw

            # Find shortest and longest words in domain and paths
            shortest_words_raw = ''
            shortest_word_host = ''
            shortest_word_path = ''
            longest_words_raw = ''
            longest_word_host = ''
            longest_word_path = ''
            try:
                 shortest_word_host = len(min(domain_words, key=len))
                 longest_word_host = len(max(domain_words, key=len))
            except:
                 shortest_word_host = 0
                 longest_word_host = 0
            features['shortest_word_host'] = shortest_word_host
            features['longest_word_host'] = longest_word_host
            try:
                 shortest_word_path = len(min(path_words, key=len))
                 longest_word_path = len(max(path_words, key=len))
            except:
                   shortest_word_path = 0
                   longest_word_path = 0
            features['longest_word_path'] = longest_word_path
            features['shortest_word_path'] = shortest_word_path
            try:
              shortest_words_raw = len(min(domain_words + path_words, key=len))
              longest_words_raw = len(max(domain_words + path_words, key=len))
            except:
                   shortest_words_raw = 0
                   longest_words_raw = 0
            features['longest_words_raw'] = longest_words_raw
            features['shortest_words_raw'] = shortest_words_raw
            # Calculate average word lengths in domain and paths
            try:
                average_word_raw = length_words_raw / (len(domain_words) + len(path_words))
            except ZeroDivisionError:
                      average_word_raw = 0
            features['average_word_raw'] = average_word_raw
            try:
                  average_word_host = len(domain) / len(domain_words)
            except ZeroDivisionError:
                     average_word_host = 0
            features['average_word_host'] = average_word_host
            try:
                average_word_path = len(path) / len(path_words)
            except ZeroDivisionError:
                      average_word_path = 0
            features['average_word_path'] = average_word_path

            # Check for phishing hints in page content
            try:
              phish_hints = ''
              phish_keywords = ['login', 'password', 'account', 'verify', 'security', 'update', 'confirm']
              soup = BeautifulSoup(response.content, 'html.parser')
              for keyword in phish_keywords:
                if soup.find(text=re.compile(keyword, re.IGNORECASE)):
                    phish_hints += keyword + ', '
              features['phish_hints'] = len(phish_hints)
            except:
                  features['phish_hints'] = 0

        """ # Check if domain is present in brand
            brand = 'example'
            domain_in_brand = domain in brand

            # Check if brand is present in subdomain or path
            subdomain, domain, suffix = tldextract.extract(url_parsed.netloc)
            brand_in_subdomain = brand in subdomain
            brand_in_path = brand in path
            # Send HTTP request to webpage
            response = requests.get(url)"""
        # Count number of hyperlinks
        try:
             hyperlinks = soup.find_all('a')
             hyperlinks_number = len(hyperlinks)
             features['hyperlinks_number'] = hyperlinks_number
        except:
             features['hyperlinks_number'] = 0
        # Calculate ratio of internal to external hyperlinks
        try:
           int_hyperlinks = []
           ext_hyperlinks = []
           for link in hyperlinks:
                   if link.get('href') and not link.get('href').startswith('#') and not link.get('href').startswith('http'):
                          int_hyperlinks.append(link)
                   elif link.get('href') and link.get('href').startswith('http'):
                                  ext_hyperlinks.append(link)
           if len(int_hyperlinks) > 0:
                      ratio_intHyperlinks = len(int_hyperlinks) / len(hyperlinks_number)
           if len(ext_hyperlinks) > 0:
                      ratio_extHyperlinks = len(ext_hyperlinks) / len(hyperlinks_number)
           features['ratio_intHyperlinks'] = ratio_intHyperlinks
           features['ratio_extHyperlinks'] = ratio_extHyperlinks
        except:
           features['ratio_intHyperlinks'] = 0
           features['ratio_extHyperlinks'] = 0
        # Count number of null hyperlinks
           try:
              null_hyperlinks = soup.find_all('a', href=None)
              ratio_nullHyperlinks = len(null_hyperlinks) / len(hyperlinks_number)
              features['ratio_nullHyperlinks'] = ratio_nullHyperlinks
           except:
                features['ratio_nullHyperlinks'] = 0
        # Count number of external CSS files
        try:
          external_css = soup.find_all('link', rel='stylesheet', href=lambda href: href and not href.startswith('#'))
          extCSS_number = len(external_css)
          features['extCSS_number'] = extCSS_number
        except:
          features['extCSS_number'] = 0
        # Submit email to haveibeenpwned API
        try:
         submit_emails = 0
         headers = {'User-Agent': 'Mozilla/5.0'}
         response = requests.get(url, headers=headers)
         if response.status_code == 200:
                     submit_emails = 1
         features['submit_emails'] = submit_emails
        except:
               features['submit_emails'] = 0
        # Count number of internal and external media files
        try:
         ratio_intMedia = 0
         ratio_extMedia = 0
         int_media = 0
         ext_media = 0
         response = requests.get(url)
         soup = BeautifulSoup(response.content, 'html.parser')
         for tag in soup.find_all(['img', 'audio', 'video']):
                 if tag.has_attr('src') and not tag['src'].startswith('http'):
                                                  int_media += 1
                 elif tag.has_attr('src') and tag['src'].startswith('http'):
                                                   ext_media += 1
         if int_media + ext_media > 0:
                  ratio_intMedia = int_media / (int_media + ext_media)
                  ratio_extMedia = ext_media / (int_media + ext_media)
         features['ratio_intMedia'] = ratio_intMedia
         features['ratio_extMedia'] = ratio_extMedia
        except:
                     features['ratio_intMedia'] = 0
                     features['ratio_extMedia'] = 0
        # Check for Same-Origin Policy violation
        try:
         sfh = ''
         response = requests.get(url, allow_redirects=False)
         if response.status_code == 200:
               soup = BeautifulSoup(response.content, 'html.parser')
               sfh_tag = soup.find('form', {'target': '_blank'})
               if sfh_tag:
                      sfh = sfh_tag.get('action')
         features['sfh'] = sfh
        except:
               features['sfh'] = 0
        # Return the feature dictionary
        return  features
        """# Calculate ratio of internal to external redirections
            int_redirections = []
            ext_redirections = []
            for tag in soup.find_all():
                    if tag.get('meta', {'http-equiv': 'refresh'}) and not tag['content'].startswith('0;url='):
                      if tag['content'].startswith('http'):
                             ext_redirections.append(tag)
                    else:
                       int_redirections.append(tag)
            if len(int_redirections) > 0:
                      ratio_intRedirection = len(int_redirections) / (len(int_redirections) + len(ext_redirections))
            if len(ext_redirections) > 0:
                      ratio_extRedirection = len(ext_redirections) / (len(int_redirections) + len(ext_redirections))

        # Calculate ratio of internal to external errors
            int_errors = []
            ext_errors = []
            for link in hyperlinks:
                    if link.get('href') and link.get('href').startswith('http'):
                                try:
                                      response = requests.get(link.get('href'))
                                      if response.status_code == 200:
                                                        continue
                                      elif response.status_code >= 400 and response.status_code < 500:

                                              ext_errors.append(link)
                                      else:
                                               continue
                                except:
                                       ext_errors.append(link)
                    elif link.get('href') and not link.get('href').startswith('#'):
                                    try:
                                        response = requests.get(url + link.get('href'))
                                        if response.status_code == 200:
                                                       continue
                                        elif response.status_code >= 400 and response.status_code < 500:
                                                  int_errors.append(link)
                                        else:
                                                 continue
                                    except:
                                        int_errors.append(link)
            if len(int_errors) > 0:
                            ratio_intErrors = len(int_errors) / (len(int_errors) + len(ext_errors))
            if len(ext_errors) > 0:
                            ratio_extErrors = len(ext_errors) / (len(int_errors) + len(ext_errors))

            # Check if login form is present
            login_form = len(soup.find_all('form', {'class': 'login'})) > 0

            # Check if external favicon is used
            external_favicon = len(soup.find_all('link', {'rel': 'icon', 'href': lambda href: href.startswith('http')})) > 0

            # Count number of links in HTML tags
            links_in_tags = len(soup.find_all(lambda tag: tag.name in ['a', 'img', 'script', 'link'] and tag.has_attr('href')))
           """

In [ ]:
gf = mailbox.mbox('/content/enron_emails.mbox')

In [ ]:
df1 = pd.DataFrame(columns=['text','lengthOfEmailId', 'noOfDotsInEmailId',
'noOfDashesInEmailId', 'noOfSpecialCharsInEmailId', 'noOfDigitsInEmailId',
'noOfSubdomainsInEmailId', 'noof_urls','length_url', 'length_hostname','ip','ip_asn',
'ip_last_analysis_date','ip_harmless','ip_malicious','ip_suspicious','ip_timeout','ip_undetected','ip_last_modification_date'
,'ip_harmless_votes','ip_malicious_votes','first_submission_date','last_analysis_date','harmless','malicious','suspicious','timeout',
'undetected','last_http_response_code', 'last_http_response_content_length', 'last_modification_date','harmless_votes','malicious_votes'
,'dots_number', 'hyphens_number', 'at_number', 'qm_number', 'and_number',
'or_number', 'eq_number','tildes_number','percent_number','slash_number',
'star_number','colon_number','comma_number','semicolon_number',
'dollar_number','space_number', 'www_number','com_number', 'http_in_path',
'https_token','ratio_digits_url','ratio_digits_host', 'punycode', 'port',
'tld_in_path','tld_in_subdomain','abnormal_subdomain','subdomains_number',
'prefix_suffix','random_domain', 'shortening_service',  'path_extension',
'redirections_number','external_redirections_number', 'length_words_raw',
'char_repeat','shortest_word_host','longest_word_host','shortest_word_path',
'longest_word_path','shortest_words_raw','longest_words_raw',
 'average_word_raw','average_word_host','average_word_path','phish_hints',
'hyperlinks_number','ratio_intHyperlinks','ratio_extHyperlinks',
'ratio_nullHyperlinks','extCSS_number','submit_emails',
'ratio_intMedia','ratio_extMedia','sfh','noof_attachements','class label'])
stringUtil = StringUtil()
for email in gf:
    emailParser = EmailParser(email)
    no_of_attachments = emailParser.get_no_of_attachments()
    emailid_features = stringUtil.process_email_address(emailParser.get_sender_email_address())
    urls_features = stringUtil.process_urls(emailParser.get_urls())
    word_dict = stringUtil.process_text(emailParser.get_email_text())
    #print(urls_features)
    new_row = pd.Series([len(word_dict),*emailid_features, *urls_features.values(),
                       no_of_attachments,0], index=df1.columns)
    df1 = df1.append(new_row, ignore_index=True)

#get most common words from phishing emails
malicious_words = stringUtil.get_most_common_words()


In [ ]:
df1.head()

In [ ]:
df1.tail()

In [ ]:
from google.colab import files
# Save the DataFrame as a CSV file
df1.to_csv('data_ff.csv', index=False)



---



In [ ]:
df1.head()

In [ ]:
df1.tail()

In [ ]:
df1.describe()